In [4]:
import pandas as pd
import numpy as np
import json, os
from pandas.io.json import json_normalize
# import missingno as msno
import seaborn as sn
from scipy import stats, sparse
import datetime as dt
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack

In [29]:
os.chdir('D:/Kaggle/Two Sigma Connect')
tr_df = pd.read_json('train.json')
te_df = pd.read_json('test.json')

In [30]:
# count number of photos

tr_df['num_photos'] = tr_df['photos'].apply(lambda x: len(x))
te_df['num_photos'] = te_df['photos'].apply(lambda x: len(x))

In [31]:
pd.set_option('max_columns', 200)
pd.set_option('max_colwidth', 500)

In [32]:
feature_to_use = ['price', 'num_photos', 'num_features', 'bedrooms', 'bathrooms']

In [33]:
fs = ['manager_id', 'building_id', 'street_address', 'display_address']

for f in fs:
    if tr_df[f].dtype == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(list(tr_df[f].values) + list(te_df[f].values))
        tr_df[f] = le.transform(list(tr_df[f].values))
        te_df[f] = le.transform(list(te_df[f].values))
        feature_to_use.append(f)

In [34]:
for index, row in tr_df.iterrows():
    
#     tr_df.loc[index, 'len_des'] = len(row['description'])

    if 'Cats Allowed' in row['features']:
        
        tr_df.loc[index, 'pets_allowed'] = 1
        
        row['features'].remove('Cats Allowed')    
        
    else:
        
        tr_df.loc[index, 'pets_allowed'] = 0       
    
    if 'Dogs Allowed' in row['features']:
        
        tr_df.loc[index, 'pets_allowed'] = 1
            
        row['features'].remove('Dogs Allowed')
        
    else:
        
        tr_df.loc[index, 'pets_allowed'] = 0
    
            
    if 'Hardwood Floors' in row['features']:       
        
        tr_df.loc[index, 'Hardwood_Floors'] = 1
        row['features'].remove('Hardwood Floors')        
    
    else:        
        
        tr_df.loc[index, 'Hardwood_Floors'] = 0
        
    if 'Dishwasher' in row['features']:        
        
        tr_df.loc[index, 'dishwasher'] = 1
        row['features'].remove('Dishwasher')
        
    else:        
        
        tr_df.loc[index, 'dishwasher'] = 0    
    
    if 'Doorman' in row['features']:        
        
        tr_df.loc[index, 'doorman_security'] = 1 
        row['features'].remove('Doorman')
        
    else:        
        
        tr_df.loc[index, 'doorman_security'] = 0        
    
    if 'Elevator' in row['features']:        
        
        tr_df.loc[index, 'elevator'] = 1
        row['features'].remove('Elevator')
        
    else:        
        
        tr_df.loc[index, 'elevator'] = 0         
    
    if 'No Fee' in row['features']:        
        
        tr_df.loc[index, 'no_fee'] = 1 
        row['features'].remove('No Fee')
        
    else:        
        
        tr_df.loc[index, 'no_fee'] = 0   
        
        
# Test set

for index, row in te_df.iterrows():
    
    if 'Cats Allowed' in row['features']:
        
        te_df.loc[index, 'pets_allowed'] = 1
        
        row['features'].remove('Cats Allowed')    
        
    else:
        
        te_df.loc[index, 'pets_allowed'] = 0       
    
    if 'Dogs Allowed' in row['features']:
        
        te_df.loc[index, 'pets_allowed'] = 1
            
        row['features'].remove('Dogs Allowed')
        
    else:
        
        te_df.loc[index, 'pets_allowed'] = 0
    
            
    if 'Hardwood Floors' in row['features']:       
        
        te_df.loc[index, 'Hardwood_Floors'] = 1
        row['features'].remove('Hardwood Floors')        
    
    else:        
        
        te_df.loc[index, 'Hardwood_Floors'] = 0
        
    if 'Dishwasher' in row['features']:        
        
        te_df.loc[index, 'dishwasher'] = 1
        row['features'].remove('Dishwasher')
        
    else:        
        
        te_df.loc[index, 'dishwasher'] = 0    
    
    if 'Doorman' in row['features']:        
        
        te_df.loc[index, 'doorman_security'] = 1 
        row['features'].remove('Doorman')
        
    else:        
        
        te_df.loc[index, 'doorman_security'] = 0        
    
    if 'Elevator' in row['features']:        
        
        te_df.loc[index, 'elevator'] = 1
        row['features'].remove('Elevator')
        
    else:        
        
        te_df.loc[index, 'elevator'] = 0         
    
    if 'No Fee' in row['features']:        
        
        te_df.loc[index, 'no_fee'] = 1 
        row['features'].remove('No Fee')
        
    else:        
        
        te_df.loc[index, 'no_fee'] = 0

In [ ]:
feature_to_use.extend(['pets_allowed', 'Hardwood_Floors','dishwasher','doorman_security','elevator','no_fee'])

In [ ]:
print feature_to_use

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# # Baseline model

# def model_fit(alg, train, test, predictors, printFeatureImportance = True):
#     alg.fit(train[predictors], train['interest_level'])
    
#     # predict on the training set 
#     train_pred = alg.predict(train[predictors])
#     test_predProb = alg.predict_proba(train[predictors])[:,1]
    
#     # Perform cross-validation
    
# #     if performCV:
# #         cv_score = cross_validation.cross_val_score(alg,train[predictors], train['interest_level'], cv=cv_folds, scoring='roc_auc')
        
#     print "\n Model Report"
#     print "Accuracy : %.4g" %metrics.accuracy_score(train['interest_level'],train_pred)
# #     print "AUC Score (Train): %f" %metrics.roc_auc_score(train['interest_level'],train_predProb)
    
# #     if performCV:
# #         print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))
    
#     if printFeatureImportance:
#         feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
#         feat_imp.plot(kind='bar', title='Feature Importances')
#         plt.ylabel('Feature Importance Score')

In [ ]:
tr_df['num_features'] = tr_df['features'].apply(lambda x: len(x))
te_df['num_features'] = te_df['features'].apply(lambda x: len(x))

In [ ]:
tr_df['features'] = tr_df['features'].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
te_df['features'] = te_df['features'].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

In [ ]:
tfidf = CountVectorizer(stop_words = 'english', max_features = 200)
tr_sparse = tfidf.fit_transform(tr_df[['features','description']])
te_sparse = tfidf.fit_transform(te_df[['features','description']])

In [ ]:
tr_X = sparse.hstack([tr_df[feature_to_use], tr_sparse]).tocsr()
te_X = sparse.hstack([te_df[feature_to_use], te_sparse]).tocsr()

In [ ]:
target_var_map = {'high':0, 'medium':1, 'low':2}
tr_y = np.array(tr_df['interest_level'].apply(lambda x : target_var_map[x]))
print (tr_X.shape, tr_y.shape)

In [ ]:
tr_X_arr = tr_X.toarray()

In [ ]:
print tr_df.groupby('interest_level')['interest_level'].count()

In [ ]:
tr_X_df = pd.SparseDataFrame([pd.SparseSeries(tr_X[i].toarray().ravel()) for i in np.arange(tr_X.shape[0])])
tr_y_df = pd.DataFrame(tr_y.ravel())

In [34]:
te_X_df = pd.SparseDataFrame([ pd.SparseSeries(te_X[i].toarray().ravel()) for i in np.arange(te_X.shape[0])])

In [ ]:
# Tuning parameters (Grid Search) - Step 1

# predictors = [x for x in tr_df.columns if x not in ['created','description','features','latitude','longitude','photos']]
param_test1 = {'n_estimators':range(20,80,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, 
                                                               min_samples_split=500,
                                                               min_samples_leaf=50,
                                                               max_depth=8,
                                                               max_features='sqrt', 
                                                               subsample=0.8,
                                                               random_state=10), 
                                                               param_grid = param_test1) 
#                                                                scoring='roc_auc',
#                                                                n_jobs=4,
#                                                                iid=False, 
#                                                                cv=5)
gsearch1.fit(tr_X_arr,tr_y)

In [ ]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
# Tune Tree Parameters - Step 1

param_test2 = {'max_depth':range(5,16,2)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, 
                                                               n_estimators = 70,
                                                               min_samples_split=500,
                                                               min_samples_leaf=50,
                                                               max_features='sqrt', 
                                                               subsample=0.8,
                                                               random_state=10), 
                                                               param_grid = param_test2)


gsearch2.fit(tr_X_arr,tr_y)

In [ ]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
# Tune Tree Parameters - Step 1

param_test3 = {'min_samples_split':range(200,1001,200)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, 
                                                               n_estimators = 70,
                                                               max_depth = 15,
                                                               min_samples_leaf = 30,
                                                               max_features='sqrt', 
                                                               subsample=0.8,
                                                               random_state=10), 
                                                               param_grid = param_test3)


gsearch3.fit(tr_X_arr,tr_y)

In [ ]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [32]:
# Tune Tree Parameters - Step 1

estimator = GradientBoostingClassifier(learning_rate = 0.05, n_estimators = 1500, max_depth = 15,
                                                               min_samples_split = 200,
                                                               min_samples_leaf = 30,
                                                               max_features = 'sqrt',
                                                               subsample=0.8,
                                                               random_state=10)


estimator.fit(tr_X_arr,tr_y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=15,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=30,
              min_samples_split=200, min_weight_fraction_leaf=0.0,
              n_estimators=1000, presort='auto', random_state=10,
              subsample=0.8, verbose=0, warm_start=False)

In [33]:
# predict the results on train set and compare with actual values

# est = gsearch3.best_estimator_
train_pred = estimator.predict(tr_X_arr)
print 'Accuracy : %.4g' %metrics.accuracy_score(tr_y, train_pred)

Accuracy : 0.8813


In [ ]:
# Perform a grid search to fine tune with the parameters 

# param_grid = {'max_depth':[4, 6, 8]}

# est = RandomForestClassifier(n_estimators = 2000)
# gs_cv = GridSearchCV(est, param_grid).fit(tr_X_arr, tr_y)
# print gs_cv.best_params_

In [ ]:
# est = RandomForestClassifier(n_estimators = 2000, max_depth = 8)
# rfc = est.fit(tr_X_arr, tr_y)
# print rfc.feature_importances_

In [ ]:
# Gradient Boosting Classifier 

# est = GradientBoostingClassifier(n_estimators = 2000, learning_rate = 0.1, max_depth = 8)
# est.fit(tr_df_X, tr_df_y)
# print est.train_score_

In [ ]:
# pred_X = est.predict(tr_df_X)
# print confusion_matrix(pred_X, tr_df_y)

In [ ]:
# te_df_pred = te_df[feature_to_use]
# print te_df_pred.head()

In [35]:
pred_te = estimator.predict_proba(te_X_df)
pred_df = pd.DataFrame(pred_te, columns = ('high', 'medium', 'low'))
print pred_df.head()

       high    medium       low
0  0.058748  0.341402  0.599850
1  0.013813  0.052648  0.933538
2  0.006410  0.080507  0.913083
3  0.105837  0.166390  0.727772
4  0.018672  0.075598  0.905730


In [36]:
pred_df['listing_id'] = te_df.listing_id.values
pred_df.to_csv('submission7_RF_2000_8.csv', index = False)

In [ ]:
# create price bins (Random)

# tr_df['price_bins'] = pd.cut(tr_df['price'], bins = 2, labels = ['Low Price', 'High Price'])
# price_map = {'Low Price':0, 'High Price':1}

# tr_df['price_bins'] = tr_df['price_bins'].apply(lambda x : price_map[x])

In [ ]:
# how the number of photos & features have affected the interest level?

# fig,(ax1, ax2) = plt.subplots(nrows = 2)
# fig.set_size_inches(13,8)

# tr_df_1 = tr_df[tr_df.num_photos <= 15]
# tr_df_stack = tr_df_1.groupby(['num_photos', 'interest_level'])['num_photos'].count().unstack('interest_level').fillna(0)

# tr_df_stack[['low', 'medium', 'high']].plot(kind = 'bar', ax = ax1)

# tr_df_stack1 = tr_df.groupby(['num_features', 'interest_level'])['num_features'].count().unstack('interest_level').fillna(0)
# tr_df_stack1[['low', 'medium', 'high']].plot(kind = 'bar', ax = ax2)


In [ ]:
# num_photos vs num_features vs interest levels

# fig, (ax1) = plt.subplots()
# fig.set_size_inches(13, 5)

# ax1.scatter(tr_df[tr_df['interest_level'] == 'low']['num_photos'], tr_df[tr_df['interest_level'] == 'low']['num_features'], c = 'green', s = 40)
# ax1.scatter(tr_df[tr_df['interest_level'] == 'medium']['num_photos'], tr_df[tr_df['interest_level'] == 'medium']['num_features'], c = 'red', s = 40)
# ax1.scatter(tr_df[tr_df['interest_level'] == 'high']['num_photos'], tr_df[tr_df['interest_level'] == 'high']['num_features'], c = 'blue', s = 80)

# ax1.set_xlabel('num_photos')
# ax1.set_ylabel('num_features')

# ax1.legend(('Low', 'Medium', 'High'), scatterpoints = 1, loc = 'upper right', fontsize = 15)

In [ ]:
# For test set 

# te_df['price_bins'] = pd.cut(te_df['price'], bins = 2, labels = ['Low Price', 'High Price'])
# price_map = {'Low Price':0, 'High Price':1}

# te_df['price_bins'] = te_df['price_bins'].apply(lambda x : price_map[x])

In [ ]:
# correlation between price and bed / bath rooms

# corr = tr_df[['building_id', 'manager_id', 'price']].corr()
# mask = np.array(corr)

# mask[np.tril_indices_from(mask)] = False
# fig,(ax) = plt.subplots()
# fig.set_size_inches = (20, 10)
# sn.heatmap(corr, mask = mask, annot = True, square = True)

In [ ]:
from wordcloud import WordCloud

In [ ]:
fig,(axes) = plt.subplots(nrows = 3, ncols = 2)
fig.set_size_inches(13, 15)

pets = tr_df.groupby(['pets_allowed','interest_level'])['pets_allowed'].count().unstack().fillna(0)
pets[['medium','high']].plot(kind = 'bar', ax = axes[0][0])

floors = tr_df.groupby(['Hardwood_Floors','interest_level'])['Hardwood_Floors'].count().unstack().fillna(0)
floors[['medium','high']].plot(kind = 'bar', ax = axes[0][1])

dishwasher = tr_df.groupby(['dishwasher','interest_level'])['dishwasher'].count().unstack().fillna(0)
dishwasher[['medium','high']].plot(kind = 'bar', ax = axes[1][0])

doorman = tr_df.groupby(['doorman_security','interest_level'])['doorman_security'].count().unstack().fillna(0)
doorman[['medium','high']].plot(kind = 'bar', ax = axes[1][1])

elevator = tr_df.groupby(['elevator','interest_level'])['elevator'].count().unstack().fillna(0)
elevator[['medium','high']].plot(kind = 'bar', ax = axes[2][0])

nofee = tr_df.groupby(['no_fee','interest_level'])['no_fee'].count().unstack().fillna(0)
nofee[['medium','high']].plot(kind = 'bar', ax = axes[2][1])

In [ ]:
text_fe = ""
for index, row in tr_df_sample.iterrows():
    for feature in row['features']:
        text_fe = " ".join([text_fe,"_".join(feature.strip().split(" "))]).strip()

plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text_fe)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for features", fontsize=30)
plt.axis("off")
plt.show()

In [ ]:
print tr_df_sample[['features', 'pets_allowed', 'Hardwood_Floors','dishwasher','doorman_security','elevator']].tail(10)

In [ ]:
test_des = ""

for index, row in tr_df.iterrows():
    for feature in row['features']:
        test_des = " ".join([test_des, row['description']])
    
test_des = test_des.strip()

plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text_des)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud)
plt.title("Wordcloud for description", fontsize=30)
plt.axis("off")
plt.show()

In [ ]:
## Predictive models 




